Практическая работа №2
Козлов Ф.С. ББМО-02-23

Выбор варианта

In [10]:
variant = 26 % 16 + 1
print(f'Вариант №{variant}')

#Вариант 11


Вариант №11


##Описание структуры полей логов и разработка парсера

Здесь я начал с анализа структуры логов и создания парсера для нескольких разных типов сообщений. Каждое сообщение содержит временную метку, уровень логирования и сам текст сообщения. Чтобы выделить эти элементы, я использовал регулярное выражение.

In [19]:
import re

log_messages = [
    "2023-10-01 14:30:45 INFO User logged in",
    "2023-10-01 14:32:00 ERROR Connection timeout",
    "2023-10-01 14:35:22 DEBUG Configuration loaded",
    "2023-10-01 14:37:10 WARNING Disk space low",
    "2023-10-01 14:38:55 INFO File saved successfully"
]

regex_pattern = r"(\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}:\\d{2}) (\\w+) (.*)"

parsed_logs = []
for log_line in log_messages:
    matches = re.match(regex_pattern, log_line)
    if matches:
        timestamp = matches.group(1)
        log_level = matches.group(2)
        message = matches.group(3)
        parsed_logs.append({'timestamp': timestamp, 'log_level': log_level, 'message': message})

for log in parsed_logs:
    print(log)


##Парсинг лога и сохранение в СУБД

После парсинга я решил сохранить данные в базе данных SQLite. Создал таблицу для хранения временной метки, уровня логирования и сообщения, затем добавил в неё записи.

In [20]:
import sqlite3

conn = sqlite3.connect('logs.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        log_level TEXT,
        message TEXT
    )
''')
conn.commit()

# Вставка данных в таблицу
for log in parsed_logs:
    cursor.execute("INSERT INTO logs (timestamp, log_level, message) VALUES (?, ?, ?)",
                   (log['timestamp'], log['log_level'], log['message']))
conn.commit()

# Проверка записи в базу данных
print("Проверка данных в базе:")
for row in cursor.execute("SELECT * FROM logs"):
    print(row)

conn.close()


Проверка данных в базе:


##Нормализация логов и сохранение в СУБД

Чтобы привести данные к единой форме, я сделал нормализацию — перевел уровень логирования в верхний регистр для унификации.

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка данных в DataFrame
conn = sqlite3.connect('logs.db')
logs_df = pd.read_sql_query("SELECT * FROM logs", conn)
conn.close()

# Проверка данных в DataFrame
print("Данные из базы данных:")
print(logs_df)

# Проверка наличия данных перед визуализацией
if not logs_df.empty:
    # График распределения уровней логов
    plt.figure(figsize=(10, 5))
    log_level_counts = logs_df['log_level'].value_counts()
    log_level_counts.plot(kind='bar')
    plt.xlabel('Log Level')
    plt.ylabel('Count')
    plt.title('Log Level Distribution')
    plt.show()

    # График количества логов по времени
    plt.figure(figsize=(10, 5))
    logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'])
    logs_df.set_index('timestamp', inplace=True)
    logs_df.resample('T').size().plot()
    plt.xlabel('Time')
    plt.ylabel('Log Count per Minute')
    plt.title('Log Count Over Time')
    plt.show()
else:
    print("Данных для отображения нет.")


Данные из базы данных:
Empty DataFrame
Columns: [id, timestamp, log_level, message]
Index: []
Данных для отображения нет.
